In [1]:
import sys
from pathlib import Path
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
sys.path.append(str(Path("../../..").resolve()))
from src.constants import model_data_dir

In [2]:
remove_cols = ["sc_1d_fwd_rel_d", "mom_1d_fwd_rel_d"]

df_train = pd.read_csv(model_data_dir / "train_classification.csv").drop(remove_cols + ["Date"], axis=1)
df_test = pd.read_csv(model_data_dir / "test_classification.csv").drop(remove_cols, axis=1)
test_dates = pd.to_datetime(df_test["Date"])
df_test.drop("Date", axis=1, inplace=True)

((y_train, x_train), (y_test, x_test)) = ((df_train.pop("value_1d_fwd_rel_d"), df_train), (df_test.pop("value_1d_fwd_rel_d"), df_test))

In [9]:
pipe = Pipeline([
    ("log_reg", LogisticRegression(penalty="elasticnet", tol=0.001, solver="saga"))
])

param_grid = {"log_reg__l1_ratio": np.arange(0, 1, 0.02)}

grid_search = GridSearchCV(pipe, 
                           param_grid, 
                           cv = 5, 
                           n_jobs=-1, # use all cores
                           scoring = "accuracy", 
                           return_train_score=True)

grid_search.fit(x_train, y_train) 

/Users/qiushi/workspace/dsi-courses/APlusBernstein-Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/qiushi/workspace/dsi-courses/APlusBernstein-Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/qiushi/workspace/dsi-courses/APlusBernstein-Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/qiushi/workspace/dsi-courses/APlusBernstein-Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/qiushi/workspace/dsi-courses/APlusBernstein-Project/venv/lib/python3.9/site-packages/

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('log_reg',
                                        LogisticRegression(penalty='elasticnet',
                                                           solver='saga',
                                                           tol=0.001))]),
             n_jobs=-1,
             param_grid={'log_reg__l1_ratio': array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
       0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
       0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
       0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
       0.88, 0.9 , 0.92, 0.94, 0.96, 0.98])},
             return_train_score=True, scoring='accuracy')

In [10]:
y_test_pred = grid_search.best_estimator_.predict(x_test)

np.mean(y_test == y_test_pred)

0.5100742311770944